# FEATURE ENGINEERING: KICKSTARTER INTERMEDIATE DATA

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
kick_inter = pd.read_csv('../../data/02_intermediate/kick_inter.csv', 
                         parse_dates=['created_at', 'deadline', 
                                      'last_update_published_at', 
                                      'launched_at', 'state_changed_at'])

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11,15,16,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cluster_features_df =  pd.read_csv('../../data/03_processed/KNN_cluster_features.csv')

In [4]:
kick_inter.columns

Index(['backers_count', 'blurb', 'converted_pledged_amount', 'country',
       'created_at', 'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'last_update_published_at', 'launched_at', 'name', 'permissions',
       'pledged', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'unread_messages_count',
       'unseen_activity_count', 'urls', 'usd_pledged', 'usd_type',
       'sub_category', 'overall_category', 'city', 'country_loc', 'state_loc',
       'creator_name', 'creator_slug'],
      dtype='object')

In [5]:
cluster_features_df.head()

,Unnamed: 0,id,cluster_predictions
0,0,1175125319,3
1,1,920424993,0
2,2,1253520992,3
3,3,335137457,2
4,4,58084093,2


## Sources

* average exchange rate over a decade - https://www.ofx.com/en-us/forex-news/historical-exchange-rates/yearly-average-rates/
* world regions https://unstats.un.org/unsd/methodology/m49/
* country codes https://laendercode.net/en/2-letter-list.html

In [6]:
pd.set_option('display.max_columns', None)

## Restrict to Successful and Failed Campaings

In [7]:
kick_inter.state.value_counts()

successful    147268
failed        129466
live           38382
canceled       16728
suspended       1055
Name: state, dtype: int64

In [8]:
kick_inter = kick_inter.loc[(kick_inter['state'] == 'successful')|(kick_inter['state'] == 'failed')]

## Features List

**FEATURES TO CREATE (COLUMNS TO KEEP)**
1. blurb - blurb word count
1. launched_at & deadline (length of campaign) 
1. currency
1. delta between created_at and launched_at 
1. sub_category (159 values) 
1. country_loc (210 values) 
1. state_loc
1. goal in USD (goal * fx_rate) 
---
1. **state [TARGET]**

In [9]:
kick_inter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276734 entries, 1 to 332898
Data columns (total 42 columns):
backers_count               276734 non-null int64
blurb                       276732 non-null object
converted_pledged_amount    164263 non-null float64
country                     276734 non-null object
created_at                  276734 non-null datetime64[ns]
currency                    276734 non-null object
currency_symbol             276734 non-null object
currency_trailing_code      276734 non-null bool
current_currency            164263 non-null object
deadline                    276734 non-null datetime64[ns]
disable_communication       276734 non-null bool
friends                     50 non-null object
fx_rate                     156437 non-null float64
goal                        276734 non-null float64
id                          276734 non-null int64
is_backing                  50 non-null object
is_starrable                172753 non-null object
is_starred       

In [10]:
kick_inter.head(3)

,backers_count,blurb,converted_pledged_amount,country,created_at,currency,currency_symbol,currency_trailing_code,current_currency,deadline,disable_communication,friends,fx_rate,goal,id,is_backing,is_starrable,is_starred,last_update_published_at,launched_at,name,permissions,pledged,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,unread_messages_count,unseen_activity_count,urls,usd_pledged,usd_type,sub_category,overall_category,city,country_loc,state_loc,creator_name,creator_slug
1,568,for Tabletop Role Playing Games like Dungeons ...,18969.0,US,2019-06-02 21:06:55,USD,$,True,USD,2019-07-18 03:55:00,False,NaN,1.0,5000.0,1175125319,NaN,False,NaN,NaT,2019-06-16 02:20:41,RPG Minimalist Creature Dice & Status / Condit...,NaN,18969.0,rpg-minimalist-creature-dice-and-status-condit...,https://www.kickstarter.com/discover/categorie...,True,False,successful,2019-07-18 03:55:01,1.0,NaN,NaN,"{""web"":{""project"":""https://www.kickstarter.com...",18969.0,domestic,Tabletop Games,games/tabletop games,Holland,US,MI,quEmpire Gaming,quempire
3,80,We have a new album that we are ready to relea...,3691.0,US,2019-06-27 18:36:40,USD,$,True,USD,2019-07-18 03:30:00,False,NaN,1.0,3500.0,920424993,NaN,False,NaN,NaT,2019-06-29 04:17:17,Drank The Gold's new album: Sipped The Silver,NaN,3691.0,drank-the-golds-new-album-sipped-the-silver,https://www.kickstarter.com/discover/categorie...,True,True,successful,2019-07-18 03:30:00,1.0,NaN,NaN,"{""web"":{""project"":""https://www.kickstarter.com...",3691.0,domestic,Music,music,Saratoga Springs,US,NY,Drank The Gold,drankthegold
6,23,Alluvia Magazine is a print publication and cr...,886.0,US,2019-07-01 22:55:44,USD,$,True,USD,2019-07-18 03:20:41,False,NaN,1.0,500.0,1253520992,NaN,False,NaN,NaT,2019-07-03 03:20:41,Alluvia Magazine: Amplifying POC environmental...,NaN,886.0,alluvia-magazine-amplifying-poc-environmental-art,https://www.kickstarter.com/discover/categorie...,True,True,successful,2019-07-18 03:20:41,1.0,NaN,NaN,"{""web"":{""project"":""https://www.kickstarter.com...",886.0,domestic,Zines,publishing/zines,Oakland,US,CA,Zahira Chaudhry,NaN


In [11]:
kick_inter.columns

Index(['backers_count', 'blurb', 'converted_pledged_amount', 'country',
       'created_at', 'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'last_update_published_at', 'launched_at', 'name', 'permissions',
       'pledged', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'unread_messages_count',
       'unseen_activity_count', 'urls', 'usd_pledged', 'usd_type',
       'sub_category', 'overall_category', 'city', 'country_loc', 'state_loc',
       'creator_name', 'creator_slug'],
      dtype='object')

In [12]:
kick_inter.drop(columns=['backers_count', 'converted_pledged_amount', 'country',
                         'currency_symbol', 'currency_trailing_code', 'current_currency', 
                         'friends', 'is_backing', 'is_starrable', 'is_starred',
                         'last_update_published_at', 'permissions', 'pledged', 
                         'slug', 'source_url', 'spotlight', 'staff_pick', 'static_usd_rate', 
                         'unread_messages_count','unseen_activity_count', 'urls', 'usd_pledged', 
                         'usd_type', 'overall_category', 'creator_name', 'creator_slug', 'disable_communication'], 
                inplace=True)

### BLURB WORD COUNT

In [13]:
kick_inter['blurb_word_count'] = kick_inter.blurb.str.split().str.len()

In [14]:
kick_inter.drop('blurb', axis=1, inplace=True)

### LENGTH OF CAMPAIGN

In [15]:
kick_inter['campaign_length'] = (kick_inter['deadline'] - kick_inter['launched_at']).dt.days

### DELTA BETWEEN CREATED AT AND LAUNCHED AT

In [16]:
kick_inter['delta_created_launched'] = (kick_inter['launched_at'] - kick_inter['created_at']).dt.days

### GOAL IN USD

Time Period

In [17]:
kick_inter.created_at.min()

Timestamp('2009-04-21 17:35:35')

In [18]:
kick_inter.created_at.max()

Timestamp('2019-07-10 23:26:41')

### CONVERT GOAL TO USD

Number of null values per currency

In [19]:
kick_inter

,created_at,currency,deadline,fx_rate,goal,id,launched_at,name,state,state_changed_at,sub_category,city,country_loc,state_loc,blurb_word_count,campaign_length,delta_created_launched
1,2019-06-02 21:06:55,USD,2019-07-18 03:55:00,1.000000,5000.0,1175125319,2019-06-16 02:20:41,RPG Minimalist Creature Dice & Status / Condit...,successful,2019-07-18 03:55:01,Tabletop Games,Holland,US,MI,9.0,32,13
3,2019-06-27 18:36:40,USD,2019-07-18 03:30:00,1.000000,3500.0,920424993,2019-06-29 04:17:17,Drank The Gold's new album: Sipped The Silver,successful,2019-07-18 03:30:00,Music,Saratoga Springs,US,NY,14.0,18,1
6,2019-07-01 22:55:44,USD,2019-07-18 03:20:41,1.000000,500.0,1253520992,2019-07-03 03:20:41,Alluvia Magazine: Amplifying POC environmental...,successful,2019-07-18 03:20:41,Zines,Oakland,US,CA,17.0,15,1
7,2019-06-10 16:52:32,USD,2019-07-18 03:00:00,1.000000,6800.0,335137457,2019-06-17 13:21:14,Fae Archaic: A Crime-ridden Fairy Tale REDUX,successful,2019-07-18 03:00:00,Graphic Novels,San Francisco,US,CA,12.0,30,6
8,2019-06-17 23:47:44,USD,2019-07-18 02:57:22,1.000000,600.0,58084093,2019-06-18 02:57:22,Old Queens of Mewni: Star Vs. the Forces of Ev...,successful,2019-07-18 02:57:22,Mixed Media,New Bedford,US,MA,19.0,30,0
11,2019-06-04 16:15:51,USD,2019-07-18 02:45:53,1.000000,5000.0,1117025715,2019-06-18 02:45:53,I have a new song to sing,successful,2019-07-18 02:45:53,Faith,Portland,US,ME,16.0,30,13
13,2019-06-18 02:09:38,USD,2019-07-18 02:14:00,1.000000,100.0,750949205,2019-06-18 03:30:25,The Kid Anders,successful,2019-07-18 02:14:00,Narrative Film,Los Angeles,US,CA,26.0,29,0
14,2019-02-12 22:39:56,USD,2019-07-18 02:08:03,1.000000,3500.0,1852138418,2019-06-18 02:08:03,Angelic Heart Ita Bag and Enamel pins,successful,2019-07-18 02:08:03,Design,Milwaukee,US,WI,5.0,30,125
17,2019-06-21 01:58:55,USD,2019-07-18 01:05:49,1.000000,1000.0,46781774,2019-07-04 01:05:49,Knights of the Slice RADICAL trading cards,successful,2019-07-18 01:05:49,Comics,Richmond,US,VA,14.0,14,12
18,2019-06-16 19:50:05,USD,2019-07-18 01:02:31,1.000000,825.0,1598130539,2019-06-18 01:02:31,"Bringer of the Dawn, Bringer of the Dusk - Ena...",successful,2019-07-18 01:02:31,Art,Lancaster,US,PA,20.0,30,1


In [20]:
bool_series = pd.isnull(kick_inter["fx_rate"])  
kick_inter[bool_series].currency.value_counts()

USD    95396
GBP    10819
EUR     4847
CAD     4616
AUD     2470
SEK      539
NZD      462
DKK      341
NOK      228
MXN      209
CHF      194
SGD      103
HKD       73
Name: currency, dtype: int64

Let's fill in the null fx columns with the average exchange rate from the last decade. Our dataset is over a decade so this should work just fine. 

In [21]:
avg_fx_rate_10yr = {
    'EUR': 1.231885, 
    'MXN': 0.068639, 
    'USD': 1.000000, 
    'GBP': 1.473741, 
    'CAD': 0.869234, 
    'SEK': 0.131576, 
    'DKK': 0.165301, 
    'HKD': 0.128529, 
    'NOK': 0.14488,
    'AUD': 0.857369, 
    'SGD': 0.757244, 
    'CHF': 1.043232, 
    'NZD': 0.744532
}
currency = ['EUR', 'MXN', 'USD', 'GBP', 'CAD', 
            'SEK', 'DKK', 'HKD', 'NOK', 'AUD', 
            'SGD', 'CHF', 'NZD']

for cur in currency:
    null_exRatesEUR = ((kick_inter.fx_rate.isnull()) & (kick_inter.currency==cur))
    kick_inter.loc[null_exRatesEUR, 'fx_rate']=avg_fx_rate_10yr[cur]

In [22]:
kick_inter['goal_usd'] = kick_inter.goal * kick_inter.fx_rate

In [23]:
kick_inter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276734 entries, 1 to 332898
Data columns (total 18 columns):
created_at                276734 non-null datetime64[ns]
currency                  276734 non-null object
deadline                  276734 non-null datetime64[ns]
fx_rate                   276734 non-null float64
goal                      276734 non-null float64
id                        276734 non-null int64
launched_at               276734 non-null datetime64[ns]
name                      276733 non-null object
state                     276734 non-null object
state_changed_at          276734 non-null datetime64[ns]
sub_category              276734 non-null object
city                      167436 non-null object
country_loc               275699 non-null object
state_loc                 275642 non-null object
blurb_word_count          276732 non-null float64
campaign_length           276734 non-null int64
delta_created_launched    276734 non-null int64
goal_usd                

In [24]:
kick_inter.drop(columns=['created_at', 'deadline', 'fx_rate', 'launched_at', 
                         'state_changed_at', 'city', 'state_loc', 'name'], 
                inplace=True)

In [25]:
kick_inter.rename({'goal':'goal_original', 'state':'target', 
                   'sub_category': 'category', 'country_loc':'country'}, 
                  axis=1, inplace=True)

### Drop Duplicated ID 

we know from our analysis that there are some IDs with several different names. When we dive in a little further it becomes clear that these are just examples of people changing the name of their campaigns lightly after launching. we will sort and in decending order by created and keep the first date. 

In [26]:
counts = kick_inter['id'].value_counts()
kick_inter[kick_inter['id'].isin(counts.index[counts == 2])]

,currency,goal_original,id,target,category,country,blurb_word_count,campaign_length,delta_created_launched,goal_usd
19053,USD,8000.0,965707427,successful,Architecture,US,14.0,15,8,8000.000000
19054,USD,8000.0,965707427,successful,Architecture,US,13.0,15,8,8000.000000
23024,CAD,550.0,790705723,successful,Comic Books,CA,24.0,14,60,413.173475
23025,CAD,550.0,790705723,successful,Comic Books,CA,24.0,14,60,415.370835
29251,EUR,25000.0,1072129063,successful,Apparel,NL,17.0,33,1,28021.942250
29252,EUR,25000.0,1072129063,successful,Apparel,NL,17.0,33,1,28098.633000
48516,USD,5000.0,387484708,successful,Product Design,US,20.0,60,6,6641.395000
48517,USD,5000.0,387484708,successful,Product Design,US,20.0,60,6,5000.000000
57743,EUR,25000.0,1409012589,successful,Apparel,NL,18.0,43,54,28526.080250
57744,EUR,25000.0,1409012589,successful,Apparel,NL,18.0,43,54,28098.633000


In [28]:
kick_inter.drop_duplicates(subset=['id'], keep='first', inplace=True)

In [29]:
len(kick_inter)

276679

In [30]:
kick_inter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276679 entries, 1 to 332898
Data columns (total 10 columns):
currency                  276679 non-null object
goal_original             276679 non-null float64
id                        276679 non-null int64
target                    276679 non-null object
category                  276679 non-null object
country                   275644 non-null object
blurb_word_count          276677 non-null float64
campaign_length           276679 non-null int64
delta_created_launched    276679 non-null int64
goal_usd                  276679 non-null float64
dtypes: float64(3), int64(3), object(4)
memory usage: 23.2+ MB


In [31]:
kick_inter.head()

,currency,goal_original,id,target,category,country,blurb_word_count,campaign_length,delta_created_launched,goal_usd
1,USD,5000.0,1175125319,successful,Tabletop Games,US,9.0,32,13,5000.0
3,USD,3500.0,920424993,successful,Music,US,14.0,18,1,3500.0
6,USD,500.0,1253520992,successful,Zines,US,17.0,15,1,500.0
7,USD,6800.0,335137457,successful,Graphic Novels,US,12.0,30,6,6800.0
8,USD,600.0,58084093,successful,Mixed Media,US,19.0,30,0,600.0


In [ ]:
kick_inter['successful_dummy'] = pd.get_dummies(kick_inter['target'], drop_first=True)

In [32]:
# we no longer need our target column anymore
kick_inter.drop(columns=['target'], inplace=True)

We also need to erase our ID column. This doesn't add any information

OK, let's take one last look out our dataset to make sure that we don't have anything else to get rid of. 

In [33]:
kick_inter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276679 entries, 1 to 332898
Data columns (total 10 columns):
currency                  276679 non-null object
goal_original             276679 non-null float64
id                        276679 non-null int64
category                  276679 non-null object
country                   275644 non-null object
blurb_word_count          276677 non-null float64
campaign_length           276679 non-null int64
delta_created_launched    276679 non-null int64
goal_usd                  276679 non-null float64
successful_dummy          276679 non-null uint8
dtypes: float64(3), int64(3), object(3), uint8(1)
memory usage: 21.4+ MB


In [34]:
kick_inter.country.isnull().sum()/len(kick_inter)

0.0037407970969968807

In [35]:
kick_inter.dropna(inplace=True)

### Let's combine the location column

In [36]:
# build a world regions dictionary 
world_regions = {
    'Northern America': ['US', 'CA', 'MX', 'GL', 'BM'], 
    'Northern Africa':['MA', 'TN', 'EG', 'LY', 'DZ', 'SD'], 
    'Eastern Africa':['KE', 'MG', 'MW', 'ET', 'UG', 'TZ', 'RW', 'MU', 'ZW', 'SO', 'SC', 'MZ', 'ZM', 'DJ', 
                      'SS'], 
    'Middle Africa':['CM', 'TD', 'CG', 'CD', 'GA', 'GQ', 'CF'], 
    'Southern Africa':['ZA', 'BW', 'SZ', 'LS'], 
    'Western Africa':['GH', 'NG', 'LR', 'SN', 'ML', 'SL', 'NE', 'BF', 'GN', 'GM', 'CI', 'CV', 'MR', 'BJ'], 
    'Caribbean':['HT', 'TT', 'PR', 'DO', 'JM', 'BS', 'CU', 'KY', 'CW', 'GP', 'SX', 'VI', 'BB', 
                 'AG', 'DM', 'LC', 'VC', 'TC', 'KN', 'GD'], 
    'Central America':['GT', 'SV','BZ', 'PA', 'HN', 'CR', 'NI'], 
    'South America':['CO', 'PE', 'VE', 'EC', 'AR', 'BR', 'CL', 'BO', 'SR', 'UY', 'PY', 'GY'], 
    'Central Asia':['KG', 'KZ', 'TJ'], 
    'Eastern Asia':['HK', 'JP', 'CN', 'MN', 'KR', 'MQ', 'TW', 'KP', 'MO'], 
    'South-eastern Asia':['ID', 'SG', 'VN', 'MY', 'TH', 'KH', 'PH', 'MM', 'LA', 'TL'], 
    'Southern Asia':['BD', 'IN', 'LK', 'NP', 'AF', 'PK', 'BT', 'IR', 'MV'], 
    'Western Asia':['IL', 'JO', 'AM', 'LB', 'AE', 'CY', 'PS', 'GE', 'YE', 'IQ', 'KW', 'AZ', 'SA', 
                    'SY', 'QA', 'BH', 'TR'], 
    'Eastern Europe':['PL', 'RO', 'UA', 'MD', 'HU', 'CZ', 'RU', 'BG', 'SK', 'BY', 'FO'], 
    'Northern Europe':['SE', 'GB', 'NO', 'IE', 'DK', 'FI', 'LT', 'IS', 'EE', 'SJ', 'LV', 'AX'], 
    'Southern Europe': ['ES', 'IT', 'HR', 'RS', 'PT', 'SI', 'GR', 'MT', 'BA', 'MK', 'XK', 'GI', 
                        'VA', 'MC', 'AL', 'ME'],
    'Western Europe':['FR', 'CH', 'DE', 'BE', 'NL', 'LU', 'AT', 'MC'], 
    'Australia and New Zealand':['AU', 'NZ'], 
    'Melanesia':['VU', 'PG', 'FJ', 'NC'], 
    'Micronesia':['FM', 'GU', 'KI'], 
    'Polynesia':['WS', 'TO', 'CK', 'PF', 'PN'], 
    'Antarctica': ['AQ']
}

In [37]:
# flip the dictionary to make each individual value in the value list a new key. 
countries_regions = {}
for key, val in world_regions.items():
    for i in range(len(val)):
        countries_regions[val[i]] = key

In [38]:
# let's make a for loop to assign the correct region to the correct country in our dataframe. 
countries = ['US', 'ES', 'FR', 'AU', 'SE', 'CH', 'CA', 'GB', 'IT', 'NO', 'HK',
       'KE', 'MX', 'IE', 'JP', 'DE', 'BE', 'NZ', 'NL', 'CN', 'CO', 'GT',
       'HR', 'PL', 'DK', 'MN', 'ID', 'LU', 'FI', 'KR', 'SG', 'GH', 'HT',
       'BD', 'RS', 'AT', 'TT', 'NG', 'PE', 'SV', 'PT', 'ZA', 'MG', 'VN',
       'MQ', 'IL', 'PR', 'RO', 'TW', 'UA', 'MD', 'JO', 'TR', 'CM', 'SI',
       'IN', 'VE', 'AM', 'EC', 'HU', 'AR', 'LK', 'MA', 'LB', 'BR', 'CL',
       'CZ', 'BO', 'GL', 'MY', 'TH', 'GR', 'MW', 'BZ', 'RU', 'AE', 'KH',
       'DO', 'CY', 'ET', 'UG', 'MT', 'PA', 'HN', 'BG', 'PH', 'LR', 'CR',
       'LT', 'TZ', 'SK', 'TN', 'WS', 'BA', 'IS', 'PS', 'BY', 'SN', 'MK',
       'KG', 'ML', 'SR', 'VU', 'PG', 'EE', 'NP', 'UY', 'GE', 'TD', 'AQ',
       'RW', 'CG', 'FM', 'SL', 'SJ', 'BW', 'TO', 'GU', 'YE', 'MM', 'AF',
       'CK', 'JM', 'CD', 'KZ', 'EG', 'MU', 'ZW', 'FJ', 'XK', 'PY', 'BS',
       'PK', 'CU', 'NI', 'KY', 'LV', 'CW', 'NE', 'GI', 'BF', 'SO', 'LY',
       'GP', 'SC', 'TJ', 'GN', 'GM', 'SX', 'MZ', 'VA', 'IQ', 'BT', 'VI',
       'KW', 'SZ', 'MC', 'CI', 'LA', 'AL', 'GY', 'BB', 'ZM', 'AG', 'DJ',
       'IR', 'DM', 'CV', 'NC', 'DZ', 'KP', 'FO', 'LS', 'TL', 'KI', 'PF',
       'MR', 'LC', 'VC', 'AX', 'MO', 'AZ', 'SA', 'SY', 'PN', 'GA', 'GQ',
       'TC', 'KN', 'QA', 'BJ', 'MV', 'SS', 'SD', 'ME', 'BH', 'GD', 'CF',
       'BM']

for country in countries:
    kick_inter.loc[kick_inter['country']==country, 'world_regions']=countries_regions[country]

In [39]:
kick_inter

,currency,goal_original,id,category,country,blurb_word_count,campaign_length,delta_created_launched,goal_usd,successful_dummy,world_regions
1,USD,5000.0,1175125319,Tabletop Games,US,9.0,32,13,5000.00000,1,Northern America
3,USD,3500.0,920424993,Music,US,14.0,18,1,3500.00000,1,Northern America
6,USD,500.0,1253520992,Zines,US,17.0,15,1,500.00000,1,Northern America
7,USD,6800.0,335137457,Graphic Novels,US,12.0,30,6,6800.00000,1,Northern America
8,USD,600.0,58084093,Mixed Media,US,19.0,30,0,600.00000,1,Northern America
11,USD,5000.0,1117025715,Faith,US,16.0,30,13,5000.00000,1,Northern America
13,USD,100.0,750949205,Narrative Film,US,26.0,29,0,100.00000,1,Northern America
14,USD,3500.0,1852138418,Design,US,5.0,30,125,3500.00000,1,Northern America
17,USD,1000.0,46781774,Comics,US,14.0,14,12,1000.00000,1,Northern America
18,USD,825.0,1598130539,Art,US,20.0,30,1,825.00000,1,Northern America


In [40]:
kick_inter.reset_index(inplace=True, drop=True)

In [43]:
kick_inter = kick_inter.merge(cluster_features_df, 
                how='left', on='id')

In [44]:
kick_inter.to_pickle('../../data/03_processed/kick_proc.pkl')

# LET'S WRITE A FUNCTION!!!

In [ ]:
# sources
# average exchange rate over a decade - https://www.ofx.com/en-us/forex-news/historical-exchange-rates/yearly-average-rates/
# world regions - https://unstats.un.org/unsd/methodology/m49/
# country codes - https://laendercode.net/en/2-letter-list.html

# restrict to successful & failed
kick_inter = kick_inter.loc[(kick_inter['state'] == 'successful')|(kick_inter['state'] == 'failed')]

# drop unneeded columns
kick_inter.drop(columns=['backers_count', 'converted_pledged_amount', 'country',
                         'currency_symbol', 'currency_trailing_code', 'current_currency', 
                         'friends', 'is_backing', 'is_starrable', 'is_starred',
                         'last_update_published_at', 'permissions', 'pledged', 
                         'slug', 'source_url', 'spotlight', 'staff_pick', 'static_usd_rate', 
                         'unread_messages_count','unseen_activity_count', 'urls', 'usd_pledged', 
                         'usd_type', 'overall_category', 'creator_name', 'creator_slug', 'disable_communication'], 
                inplace=True)

# create feature with blurb word count
kick_inter['blurb_word_count'] = kick_inter.blurb.str.split().str.len()
# drop original blurb column 
kick_inter.drop('blurb', axis=1, inplace=True)

# create length of campaign feature
kick_inter['campaign_length'] = (kick_inter['deadline'] - kick_inter['launched_at']).dt.days

# create delta between created and launched date
kick_inter['delta_created_launched'] = (kick_inter['launched_at'] - kick_inter['created_at']).dt.days

# CONVERT GOAL TO GOAL_USD 
# boolean series that are missing the exchange rate
bool_series = pd.isnull(kick_inter["fx_rate"])  
kick_inter[bool_series].currency.value_counts()

# let's fill in the null fx columnn 
avg_fx_rate_10yr = {
    'EUR': 1.231885, 
    'MXN': 0.068639, 
    'USD': 1.000000, 
    'GBP': 1.473741, 
    'CAD': 0.869234, 
    'SEK': 0.131576, 
    'DKK': 0.165301, 
    'HKD': 0.128529, 
    'NOK': 0.14488,
    'AUD': 0.857369, 
    'SGD': 0.757244, 
    'CHF': 1.043232, 
    'NZD': 0.744532
}
currency = ['EUR', 'MXN', 'USD', 'GBP', 'CAD', 
            'SEK', 'DKK', 'HKD', 'NOK', 'AUD', 
            'SGD', 'CHF', 'NZD']

for cur in currency:
    null_exRatesEUR = ((kick_inter.fx_rate.isnull()) & (kick_inter.currency==cur))
    kick_inter.loc[null_exRatesEUR, 'fx_rate']=avg_fx_rate_10yr[cur]
    
# create goal USD
kick_inter['goal_usd'] = kick_inter.goal * kick_inter.fx_rate

# drop unneeded columns 
kick_inter.drop(columns=['created_at', 'deadline', 'fx_rate', 'launched_at', 
                         'state_changed_at', 'city', 'state_loc', 'name'], 
                inplace=True)

# rename columns 
kick_inter.rename({'goal':'goal_original', 'state':'target', 
                   'sub_category': 'category', 'country_loc':'country'}, 
                  axis=1, inplace=True)

# drop duplicated ids
kick_inter.drop_duplicates(subset=['id'], keep='first', inplace=True)

kick_inter['successful_dummy'] = pd.get_dummies(kick_inter['target'], drop_first=True)

# we no longer need our target column anymore
kick_inter.drop(columns=['target'], inplace=True)

# let's drop our remaining null columns
kick_inter.dropna(inplace=True)

# LET'S COMBINE OUR COUNTRIES INTO WORLD REGIONS
# build a world regions dictionary 
world_regions = {
    'Northern America': ['US', 'CA', 'MX', 'GL', 'BM'], 
    'Northern Africa':['MA', 'TN', 'EG', 'LY', 'DZ', 'SD'], 
    'Eastern Africa':['KE', 'MG', 'MW', 'ET', 'UG', 'TZ', 'RW', 'MU', 'ZW', 'SO', 'SC', 'MZ', 'ZM', 'DJ', 
                      'SS'], 
    'Middle Africa':['CM', 'TD', 'CG', 'CD', 'GA', 'GQ', 'CF'], 
    'Southern Africa':['ZA', 'BW', 'SZ', 'LS'], 
    'Western Africa':['GH', 'NG', 'LR', 'SN', 'ML', 'SL', 'NE', 'BF', 'GN', 'GM', 'CI', 'CV', 'MR', 'BJ'], 
    'Caribbean':['HT', 'TT', 'PR', 'DO', 'JM', 'BS', 'CU', 'KY', 'CW', 'GP', 'SX', 'VI', 'BB', 
                 'AG', 'DM', 'LC', 'VC', 'TC', 'KN', 'GD'], 
    'Central America':['GT', 'SV','BZ', 'PA', 'HN', 'CR', 'NI'], 
    'South America':['CO', 'PE', 'VE', 'EC', 'AR', 'BR', 'CL', 'BO', 'SR', 'UY', 'PY', 'GY'], 
    'Central Asia':['KG', 'KZ', 'TJ'], 
    'Eastern Asia':['HK', 'JP', 'CN', 'MN', 'KR', 'MQ', 'TW', 'KP', 'MO'], 
    'South-eastern Asia':['ID', 'SG', 'VN', 'MY', 'TH', 'KH', 'PH', 'MM', 'LA', 'TL'], 
    'Southern Asia':['BD', 'IN', 'LK', 'NP', 'AF', 'PK', 'BT', 'IR', 'MV'], 
    'Western Asia':['IL', 'JO', 'AM', 'LB', 'AE', 'CY', 'PS', 'GE', 'YE', 'IQ', 'KW', 'AZ', 'SA', 
                    'SY', 'QA', 'BH', 'TR'], 
    'Eastern Europe':['PL', 'RO', 'UA', 'MD', 'HU', 'CZ', 'RU', 'BG', 'SK', 'BY', 'FO'], 
    'Northern Europe':['SE', 'GB', 'NO', 'IE', 'DK', 'FI', 'LT', 'IS', 'EE', 'SJ', 'LV', 'AX'], 
    'Southern Europe': ['ES', 'IT', 'HR', 'RS', 'PT', 'SI', 'GR', 'MT', 'BA', 'MK', 'XK', 'GI', 
                        'VA', 'MC', 'AL', 'ME'],
    'Western Europe':['FR', 'CH', 'DE', 'BE', 'NL', 'LU', 'AT', 'MC'], 
    'Australia and New Zealand':['AU', 'NZ'], 
    'Melanesia':['VU', 'PG', 'FJ', 'NC'], 
    'Micronesia':['FM', 'GU', 'KI'], 
    'Polynesia':['WS', 'TO', 'CK', 'PF', 'PN'], 
    'Antarctica': ['AQ']
}

# flip the dictionary to make each individual value in the value list a new key. 
countries_regions = {}
for key, val in world_regions.items():
    for i in range(len(val)):
        countries_regions[val[i]] = key
        

# let's make a for loop to assign the correct region to the correct country in our dataframe. 
countries = ['US', 'ES', 'FR', 'AU', 'SE', 'CH', 'CA', 'GB', 'IT', 'NO', 'HK',
       'KE', 'MX', 'IE', 'JP', 'DE', 'BE', 'NZ', 'NL', 'CN', 'CO', 'GT',
       'HR', 'PL', 'DK', 'MN', 'ID', 'LU', 'FI', 'KR', 'SG', 'GH', 'HT',
       'BD', 'RS', 'AT', 'TT', 'NG', 'PE', 'SV', 'PT', 'ZA', 'MG', 'VN',
       'MQ', 'IL', 'PR', 'RO', 'TW', 'UA', 'MD', 'JO', 'TR', 'CM', 'SI',
       'IN', 'VE', 'AM', 'EC', 'HU', 'AR', 'LK', 'MA', 'LB', 'BR', 'CL',
       'CZ', 'BO', 'GL', 'MY', 'TH', 'GR', 'MW', 'BZ', 'RU', 'AE', 'KH',
       'DO', 'CY', 'ET', 'UG', 'MT', 'PA', 'HN', 'BG', 'PH', 'LR', 'CR',
       'LT', 'TZ', 'SK', 'TN', 'WS', 'BA', 'IS', 'PS', 'BY', 'SN', 'MK',
       'KG', 'ML', 'SR', 'VU', 'PG', 'EE', 'NP', 'UY', 'GE', 'TD', 'AQ',
       'RW', 'CG', 'FM', 'SL', 'SJ', 'BW', 'TO', 'GU', 'YE', 'MM', 'AF',
       'CK', 'JM', 'CD', 'KZ', 'EG', 'MU', 'ZW', 'FJ', 'XK', 'PY', 'BS',
       'PK', 'CU', 'NI', 'KY', 'LV', 'CW', 'NE', 'GI', 'BF', 'SO', 'LY',
       'GP', 'SC', 'TJ', 'GN', 'GM', 'SX', 'MZ', 'VA', 'IQ', 'BT', 'VI',
       'KW', 'SZ', 'MC', 'CI', 'LA', 'AL', 'GY', 'BB', 'ZM', 'AG', 'DJ',
       'IR', 'DM', 'CV', 'NC', 'DZ', 'KP', 'FO', 'LS', 'TL', 'KI', 'PF',
       'MR', 'LC', 'VC', 'AX', 'MO', 'AZ', 'SA', 'SY', 'PN', 'GA', 'GQ',
       'TC', 'KN', 'QA', 'BJ', 'MV', 'SS', 'SD', 'ME', 'BH', 'GD', 'CF',
       'BM']

for country in countries:
    kick_inter.loc[kick_inter['country']==country, 'world_regions']=countries_regions[country]

# let's reset the index. 
kick_inter.reset_index(inplace=True, drop=True)

# let's merge the new cluster feature with this. 
kick_inter = kick_inter.merge(cluster_features_df, 
                how='left', on='id')
    